In [39]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from time import time
import math
from tqdm import tqdm_notebook

In [40]:
class review:
    def __init__(self, pn, name, age, ftype, sex, point, comment):
        self.pn = pn
        self.name = name
        self.age = age
        self.ftype = ftype
        self.sex = sex
        self.point = point
        self.comment = comment
    
    def __repr__(self):
        return '{},{},{},{},{},{}'.format(self.pn, self.name, self.age, self.ftype, self.point, self.comment)

class ingredient():
    def __init__(self, pn, kname, ename, exp, level, degree):
        self.pn = pn
        self.kname = kname
        self.ename = ename
        self.exp = exp
        self.level = level
        self.degree = degree
        
    def __repr__(self):
        return '{},{},{},{},{},{}'.format(self.pn, self.kname, self.ename, self.exp, self.level, self.degree)

def crawl_ingredient(pn):
    # 화장품성분  
    for n in range(5):
        info = driver.find_elements_by_css_selector('section.section-list-item.ingredient.moreinfo div button')
        sleep(1)
        if len(info)>0:
            info[0].click()
            break
            
    if len(info) == 0:
        return []
            
    measures = driver.find_element_by_css_selector('section.side-info.ingredient div.list-ingredient')
    measures = measures.find_elements_by_class_name('list')

    ings =[]
    for m in measures:
        soup = BeautifulSoup(m.get_attribute('innerHTML'), 'lxml')
        icon = soup.select_one('div.list-item > span')
        icontxt = icon.text
        icon = icon['class'][2]

        names = soup.select('div.list-item div.txt p')
        narr = names[0].text.split('\n')
        n1 = narr[0].strip()
        n2 = narr[1].strip()
        n3 = names[1].text.strip() 
        #print(icontxt, icon)
        ing = ingredient(pn, n1, n2, n3, icontxt, icon)
        ings.append(ing)
    return ings

def crawl_reviews(pn):
    driver.get('https://www.glowpick.com/product/{}'.format(pn))    
    
    uf = 'https://www.glowpick.com/product/{}?rating={}&order={}'
    
    count = '#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-score.side-right > section > div > div.side-right > ul > li > div > p'
    cnts = [int(cnt.text) for cnt in driver.find_elements_by_css_selector(count)]

    count = '#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-score.side-right > section > div > div.side-right > ul > li:nth-child({}) > div > p'
    creviews = [] 
    
    exp = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > p').text

    tag = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > ul').text
    
    pdic = {'gpa-best': 5, 'gpa-good': 4, 'gpa-soso': 3, 'gpa-bad': 2, 'gpa-worst': 1}
    #print(cnts)
    for cnt, fn1, cidx in zip(cnts, range(5, 0, -1), range(1, 6)): 
        if cnt == 0:
            #print(cnt, fn1, cidx)
            continue
                
        loop = min(math.ceil(cnt/20), 5)
        
        noloop1 = noloop2 = False
        if cnt <= 100: noloop1 = True
        if cnt <= 200: noloop2 = True
            
        for idx, fn2 in enumerate(['create_date_asc','create_date_desc','like_desc','like_asc']):
            url = uf.format(pn, fn1, fn2)
            driver.get(url)
            
            # 
            while(True):
                rlist = driver.find_elements_by_class_name('review-list-wrap')
                if len(rlist)>0:
                    if (rlist[0].text != ''):
                        body = driver.find_element_by_css_selector('body')
                        body.click()
                        break 
            
            banners = driver.find_elements_by_xpath('//*[@id="top-downloadbanner"]/button')
            if len(banners)>0:
                banners[0].click()
                
            # 
            while(True):
                ctxt = int(driver.find_element_by_css_selector(count.format(cidx)).text)
                if cnt == ctxt:
                    break
                else:
                    #print(cnt, ctxt)
                    driver.get(url) 
                        
            st = time()
            while(True):
                try:
                    body.send_keys(Keys.END)
                
                    reviews = driver.find_elements_by_css_selector('ul.review-list-wrap li div.list-item') 

                    if time() - st > 3:
                        body.send_keys(Keys.PAGE_UP)
                        sleep(0.1) 
                        body.send_keys(Keys.END)
                        st = time()

                    if len(reviews) > (loop-1) * 20:
                        break
                
                except Exception as e:
                    body = driver.find_element_by_css_selector('body')
                    body.click()
                 
            #print(len(reviews))
                
            for r in reviews:
                soup = BeautifulSoup(r.get_attribute('innerHTML'), 'lxml')
                user = soup.select_one('.user .user-info')
                name = user.select_one('.name').text
                agetype = user.select_one('.info .txt')
                arr = agetype.text.split('·')
                age = int(arr[0].replace('세',''))
                ftype = arr[1] 
                sex = agetype.select_one('span')['class']
                point = user.select_one('.info .label span')['class'][1]
                point = pdic[point]

                comment = soup.select_one('.review').text

                cr = review(pn, name, age, ftype, sex, point, comment)
                creviews.append(cr)
            
            if noloop1: break 
            if noloop2 & (idx >= 1):break

    rank = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-top > li.section-list-img.side-left > div > section > ul > li > span').text
    #print(rank) 
    
    ing = crawl_ingredient(pn)
    
    return exp, tag, creviews, ing

In [41]:
df = pd.read_csv('data/rank.csv')
df.head()

,cate,cid,rank,change,diff,pn,company,name,volume,price
0,미스트,1,1,NaN,NaN,95080,홀리카홀리카,굳세라 수퍼 세라마이드 미스트,120ml,"15,000원"
1,미스트,1,2,NaN,NaN,27484,한율,어린쑥 수분 진정수,150ml,"25,000원"
2,미스트,1,3,NaN,NaN,51533,CNP(차앤박),프로폴리스 앰플 미스트,100ml,"15,000원"
3,미스트,1,4,NaN,NaN,58165,비어썸(Beausome),오우썸 하이드레이팅 오가닉 더블미스트,120ml,"22,000원"
4,미스트,1,5,NaN,NaN,99864,보타닉힐 보,더마 인텐시브 판테놀 크림 미스트,120ml,"15,000원"


In [37]:
driver = webdriver.Chrome()

In [26]:
%time res = crawl_reviews(pn)

* 스킨 앤 굳세라 울트라 에센스 미스트 리뉴얼 제품입니다. 피부 각질 케어와 함께 민감한 피부를 진정하고, 기초 보습도를 높여주는 세라마이드 미스트입니다. 언제 어디서나 손쉽게 피부 속부터 겉까지 수분을 채워주는 #안개분사 미스트 안개분사
영양공급
저자극
피부결정돈
피부진정
CPU times: user 1.21 s, sys: 45.4 ms, total: 1.25 s
Wall time: 30.4 s


In [42]:
reviw_res = []
for idx in tqdm_notebook(df.index):
    pn = df.loc[idx, 'pn']
    res = crawl_reviews(pn)
    reviw_res.append(res)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


In [ ]:
grs = []
for gr in reviews: 
    grs.append([gr.pn, gr.name, gr.age, gr.ftype, gr.point, gr.comment])
rdf = pd.DataFrame(grs, columns=['pn','author','age','type','point','ment'])
rdf = rdf[~rdf['ment'].duplicated()]
len(rdf)

In [ ]:
gin = []
for gi in ingredients:
    gin.append([gi.pn, gi.kname, gi.ename, gi.exp, gi.level, gi.degree])
gdf = pd.DataFrame(gin, columns=['pn','name_k','name_k','exp','degree','level'])